# 测试使用sindy求解单摆

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import pandas as pd

# 物理参数
g = 9.81      # 重力加速度 (m/s^2)
L = 1.0       # 摆长 (m)
theta0 = np.pi/4  # 初始角度 (rad)
omega0 = 0.0  # 初始角速度 (rad/s)

# 定义微分方程
def pendulum_ode(t, state):
    theta, omega = state
    return [omega, - (g / L) * np.sin(theta)]

# 初始状态与时间点
y0 = [theta0, omega0]
t_eval = np.linspace(0, 10, 1000)

# 求解 ODE
sol = solve_ivp(pendulum_ode, (0, 10), y0, t_eval=t_eval)

# 计算摆球的笛卡尔坐标
theta = sol.y[0]
x = L * np.sin(theta)
y = -L * np.cos(theta)
# 提取结果
t = sol.t
theta = sol.y[0]
omega = sol.y[1]

# 保存到 DataFrame 并导出 CSV
df = pd.DataFrame({
    'time_s':    t,
    'theta_rad': theta,
    'omega_rad_s': omega
})
csv_path = 'pendulum_states.csv'
df.to_csv(csv_path, index=False)
print(f"数据已保存到：{csv_path}")
# 绘制摆球轨迹并标记起点/终点
plt.figure(figsize=(6, 6))
plt.plot(x, y, linewidth=1.5, label='trajectory')
plt.scatter(x[0], y[0],   s=50, marker='o', color='green', label='start')
plt.scatter(x[-1], y[-1], s=80, marker='x', color='red',   label='terminal')
plt.title('Pendulum Bob Trajectory with Start/End')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.legend()
plt.grid(True)
plt.axis('equal')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import pysindy as ps
from pysindy.feature_library import GeneralizedLibrary, PolynomialLibrary, FourierLibrary

# 1. 生成训练数据：真实单摆
g = 9.81
L = 1.0
def pendulum_ode(t, state):
    theta, omega = state
    return [omega, - (g/L)*np.sin(theta)]

# 初始条件与采样
theta0, omega0 = 0.2, 0.0
t_final = 10.0
n_samples = 1000
t = np.linspace(0, t_final, n_samples)
sol = solve_ivp(pendulum_ode, (0, t_final), [theta0, omega0], t_eval=t)
data = np.vstack((sol.y[0], sol.y[1])).T  # shape (n_samples, 2)

# 2. 构建并拟合 SINDy 模型
#    - 用多项式库（degree=5）来捕捉 sin(theta) 展开
#    - 用 STLSQ 稀疏回归
poly_lib = PolynomialLibrary(degree=1)            # 包含 [1, θ, ω]
fourier_lib = FourierLibrary(n_frequencies=1)      # 包含 [sin(θ), cos(θ), sin(ω), cos(ω)]
library = GeneralizedLibrary(libraries=[poly_lib, fourier_lib])
optimizer   = ps.STLSQ(threshold=0.1)
model = ps.SINDy(feature_library=library, optimizer=optimizer)
model.fit(data, t=t[1]-t[0], multiple_trajectories=False)
model.print()  # 打印识别出的方程

# 3. 用识别出的模型来模拟
#    初始状态同真实系统
sim_t = np.linspace(0, t_final, n_samples)
sim_data = model.simulate([theta0, omega0], sim_t)

# 4. 比较真值 vs SINDy 轨迹
plt.figure(figsize=(8,4))
plt.plot(t, data[:,0], 'k',    label='真实 θ(t)')
plt.plot(sim_t, sim_data[:,0],'r--', label='SINDy 拟合 θ(t)')
plt.xlabel('Time (s)')
plt.ylabel('Angle (rad)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from pysr import PySRRegressor
import matplotlib.pyplot as plt


# 1. 生成单摆数据
g, L = 9.81, 1.0
def pendulum_ode(t, state):
    theta, omega = state
    return [omega, - (g/L)*np.sin(theta)]

t_final, n_samples = 10.0, 1000
t = np.linspace(0, t_final, n_samples)
sol = solve_ivp(pendulum_ode, (0, t_final), [0.2, 0.0], t_eval=t)
theta, omega = sol.y

# 2. 估计导数
dt = t[1] - t[0]
dtheta = np.gradient(theta, dt)
domega = np.gradient(omega, dt)

# 目标输入

X = np.stack([theta,omega],axis=0).T

# 目标输出
Y = np.stack([dtheta,domega],axis=0).T

print(X[:5],Y[:5])


# 定义 PySR 模型
model = PySRRegressor(
    niterations=100,  # 迭代次数
    populations=30,  # 种群数量
    binary_operators=["+", "-", "*", "/"],  # 二元操作符
    unary_operators=["sin", "cos", "square"],  # 一元操作符
    elementwise_loss="L2DistLoss()",  # 损失函数
    parallelism='multithreading',  # 启用多线程
    verbosity=1,  # 显示详细日志 (verbosity 而不是 verbose)
)

# 拟合模型
print("Fitting model...")
model.fit(X, Y)

# 输出最佳表达式
for i, expr in enumerate(model.equations_):
    print(f"Best Expression for output {i+1}: {expr}")



Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
[[ 0.2         0.        ]
 [ 0.19990237 -0.01950584]
 [ 0.19960956 -0.0389929 ]
 [ 0.19912185 -0.05844239]
 [ 0.19843973 -0.07783555]] [[-0.00975369 -1.94863351]
 [-0.01950267 -1.94769542]
 [-0.03898657 -1.94488045]
 [-0.05843299 -1.94019046]
 [-0.07782312 -1.93363097]]
Fitting model...
Compiling Julia backend...


c:\anacoda\envs\pytorch\Lib\site-packages\pysr\sr.py:2737: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\anacoda\envs\pytorch\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 2.280e+05
Progress: 1405 / 6000 total iterations (23.417%)
════════════════════════════════════════════════════════════════════════════════════════════════════
Best equations for output 1
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           1.961e-01  1.594e+01  y₀ = -0.00022682
2           5.050e-04  5.962e+00  y₀ = sin(x₁)
3           3.873e-07  7.173e+00  y₀ = x₁ * 1.0001
5           3.873e-07  8.673e-06  y₀ = (-2.4983e-06 - x₁) * -1.0001
7           3.873e-07  1.252e-06  y₀ = ((0.72696 - x₁) * -1.0001) - -0.72705
8           3.867e-07  1.448e-03  y₀ = 1.0001 * (x₁ / cos(x₀ * 0.10139))
9           3.865e-07  4.418e-04  y₀ = (x₁ * 1.0001) / cos(square(x₀ * 0.84151))
10          3.863e-07  7.246e-04  y₀ = x₁ / cos((0.12117 - (x₀ * x₁)) * x₀)
11          3.862e-07  7.487e-05  y₀ = x₁ / cos((0.12117 - (x₀ * sin(x₁))) * x₀)
12          3.857e-07  1.440e

[ Info: Final populations:
[ Info: Output 1:
[ Info: Output 2:
[ Info: Results saved to:


Best Expression for output 1:     complexity          loss  \
0            1  3.905782e-07   
1            3  3.872708e-07   
2            5  3.872639e-07   
3            7  3.869881e-07   
4            8  3.867009e-07   
5            9  3.864223e-07   
6           10  3.862510e-07   
7           11  3.862221e-07   
8           12  3.856483e-07   
9           13  3.851600e-07   
10          14  3.851346e-07   
11          15  3.840228e-07   
12          16  3.840139e-07   
13          17  3.835609e-07   
14          19  3.824905e-07   
15          20  3.823679e-07   
16          21  3.821527e-07   
17          23  3.818796e-07   
18          24  3.818786e-07   
19          27  3.818367e-07   

                                             equation     score  \
0                                                  x1  0.000000   
1                                        x1 * 1.00013  0.004252   
2                     (-2.4998521e-6 - x1) * -1.00013  0.000009   
3                    x1 / cos